In [3]:
import pandas as pd
import re
import numpy as np
from bs4 import BeautifulSoup
import souper_v2

In [4]:
# Danh sách các giải đấu
URLs_transfermkt = {
                 'Premier League (ENG)':'https://www.transfermarkt.co.uk/premier-league/startseite/wettbewerb/GB1',
                 'Bundesliga (GER)': 'https://www.transfermarkt.co.uk/bundesliga/startseite/wettbewerb/L1',
                 'La Liga (SPA)': 'https://www.transfermarkt.co.uk/laliga/startseite/wettbewerb/ES1',
                 'Serie A (ITA)': 'https://www.transfermarkt.co.uk/serie-a/startseite/wettbewerb/IT1',
                 'Ligue 1 (FRA)': 'https://www.transfermarkt.co.uk/ligue-1/startseite/wettbewerb/FR1',
                 'Eredivise (NET)': 'https://www.transfermarkt.co.uk/eredivisie/startseite/wettbewerb/NL1',
                 'Brasileiro (BRA)': 'https://www.transfermarkt.co.uk/campeonato-brasileiro-serie-a/startseite/wettbewerb/BRA1',
                 'Liga NOS (POR)': 'https://www.transfermarkt.co.uk/liga-nos/startseite/wettbewerb/PO1',
                 'Liga MX (MEX)': 'https://www.transfermarkt.co.uk/liga-mx-clausura/startseite/wettbewerb/MEX1',
                 'Premier Liga (RUS)': 'https://www.transfermarkt.co.uk/premier-liga/startseite/wettbewerb/RU1',
                 'Championship (ENG)': 'https://www.transfermarkt.co.uk/championship/startseite/wettbewerb/GB2',
                 'Super Lig (TUR)': 'https://www.transfermarkt.co.uk/super-lig/startseite/wettbewerb/TR1',
                 'Bundesliga (AUT)': 'https://www.transfermarkt.co.uk/bundesliga/startseite/wettbewerb/A1',
                 'MLS (USA)': 'https://www.transfermarkt.co.uk/major-league-soccer/startseite/wettbewerb/MLS1',
                 'Primera División (ARG)': 'https://www.transfermarkt.co.uk/primera-division/startseite/wettbewerb/AR1N',
                 'J-League (JPN)': 'https://www.transfermarkt.co.uk/j1-league/startseite/wettbewerb/JAP1',
                 'Saudi Pro League (KSA)': 'https://www.transfermarkt.co.uk/saudi-pro-league/startseite/wettbewerb/SA1',
                 'K-League 1 (KOR)': 'https://www.transfermarkt.co.uk/k-league-1/startseite/wettbewerb/RSK1',
                 'DStv Premiership (ZAF)': 'https://www.transfermarkt.co.uk/dstv-premiership/startseite/wettbewerb/SFA1',
                 'A-League (AUS)': 'https://www.transfermarkt.co.uk/a-league-men/startseite/wettbewerb/AUS1'
                 }

In [5]:
sp = souper_v2.souper()

In [1]:
def extract_teams_or_players(soup, league_or_team, team):
    
    returner_df = pd.DataFrame()
    links = []
    teams_or_players = []
    league_or_team_list = []
    
    if team:
        team_pattern = re.compile(r'(?<=\ alt=").+(?=" class)')
        link_pattern = re.compile(r'(?<=\ href=").+[startseite].+(?=/saison_id/)')

        for each in soup.find('table', class_ = "items").find_all('a'): #, attrs={'class':'vereinprofil_tooltip tooltipstered'}
            
            if 'startseite' in str(each):

                try:
                    team_or_player = re.findall(team_pattern, str(each))[0]
                    link = re.findall(link_pattern, str(each))[0]
                    team_or_player = team_or_player.strip()
                    link = link.strip()
                    teams_or_players.append(team_or_player)
                    links.append(link)
                    league_or_team_list.append(league_or_team)

                except:
                    None
        
    else:
        player_pattern = re.compile(r'(?<=\">).+(?=</a>)')
        link_pattern = re.compile(r'(?<=\ href=").+(?=" id=)')
        
        table = soup.find("table", class_ = "items")

        for each in table.find_all('a'): #, attrs={'class': 'spielprofil_tooltip tooltipstered'}

            try:
                team_or_player = re.findall(player_pattern, str(each))[0]
                link = re.findall(link_pattern, str(each))[0]
                team_or_player = team_or_player.strip()
                link = link.strip()
                teams_or_players.append(team_or_player)
                links.append(link)
                league_or_team_list.append(league_or_team)

            except:
                None
    
    if team:
        returner_df['league'] = league_or_team_list
        returner_df['teams'] = teams_or_players
    else:
        returner_df['team'] = league_or_team_list
        returner_df['player'] = teams_or_players
        
    returner_df['link'] = links
    returner_df = returner_df.drop_duplicates(subset='link', keep="last")

    return returner_df

In [9]:
def get_teams(URLs_dict, team=True):
    
    teams_detail_df = pd.DataFrame()
    
    for each in range(len(URLs_dict.keys())):
        soup = sp.get_soup(list(URLs_dict.values())[each])
        league = list(URLs_dict.keys())[each]
        teams = extract_teams_or_players(soup, league, team)
        print(teams)
        teams_detail_df = pd.concat([teams_detail_df, teams], axis=0, sort=True)
        
        print("Complete " + league)
        
    return teams_detail_df

In [10]:
teams_transfermkt = get_teams(URLs_transfermkt)

                  league                       teams  \
4   Premier League (ENG)           Manchester United   
7   Premier League (ENG)             West Ham United   
8   Premier League (ENG)              Leicester City   
9   Premier League (ENG)                 Aston Villa   
10  Premier League (ENG)     Wolverhampton Wanderers   
11  Premier League (ENG)              Southampton FC   
12  Premier League (ENG)  Brighton &amp; Hove Albion   
13  Premier League (ENG)                  Everton FC   
14  Premier League (ENG)           Nottingham Forest   
16  Premier League (ENG)                Leeds United   
17  Premier League (ENG)              Crystal Palace   
18  Premier League (ENG)                   Fulham FC   
19  Premier League (ENG)             AFC Bournemouth   
22  Premier League (ENG)                Brentford FC   
23  Premier League (ENG)                Liverpool FC   
24  Premier League (ENG)            Newcastle United   
26  Premier League (ENG)             Manchester 

           league                 teams  \
1   Ligue 1 (FRA)             AS Monaco   
2   Ligue 1 (FRA)      Stade Rennais FC   
3   Ligue 1 (FRA)   Olympique Marseille   
4   Ligue 1 (FRA)              OGC Nice   
7   Ligue 1 (FRA)               RC Lens   
8   Ligue 1 (FRA)             FC Nantes   
9   Ligue 1 (FRA)       Montpellier HSC   
12  Ligue 1 (FRA)            FC Lorient   
13  Ligue 1 (FRA)           FC Toulouse   
14  Ligue 1 (FRA)          ESTAC Troyes   
15  Ligue 1 (FRA)     Stade Brestois 29   
16  Ligue 1 (FRA)            Angers SCO   
17  Ligue 1 (FRA)            AJ Auxerre   
18  Ligue 1 (FRA)      Clermont Foot 63   
19  Ligue 1 (FRA)            AC Ajaccio   
21  Ligue 1 (FRA)        Olympique Lyon   
22  Ligue 1 (FRA)            LOSC Lille   
23  Ligue 1 (FRA)           Stade Reims   
24  Ligue 1 (FRA)  RC Strasbourg Alsace   
29  Ligue 1 (FRA)   Paris Saint-Germain   

                                             link  
1                /as-monaco/startseite/verei

           league                  teams  \
5   Liga MX (MEX)          Santos Laguna   
8   Liga MX (MEX)           Club León FC   
10  Liga MX (MEX)              FC Juárez   
11  Liga MX (MEX)           Club Tijuana   
13  Liga MX (MEX)   Atlético de San Luis   
14  Liga MX (MEX)            Club Necaxa   
15  Liga MX (MEX)              Puebla FC   
16  Liga MX (MEX)            Mazatlán FC   
17  Liga MX (MEX)           Querétaro FC   
18  Liga MX (MEX)             CF América   
19  Liga MX (MEX)           CF Monterrey   
20  Liga MX (MEX)      Atlas Guadalajara   
21  Liga MX (MEX)             UNAM Pumas   
22  Liga MX (MEX)       Deportivo Toluca   
23  Liga MX (MEX)            Tigres UANL   
24  Liga MX (MEX)  Deportivo Guadalajara   
26  Liga MX (MEX)           CD Cruz Azul   
27  Liga MX (MEX)             CF Pachuca   

                                             link  
5           /santos-laguna/startseite/verein/1403  
8            /club-leon-fc/startseite/verein/4941  
10     

              league                  teams  \
1   Bundesliga (AUT)          SK Sturm Graz   
2   Bundesliga (AUT)                   LASK   
6   Bundesliga (AUT)             SCR Altach   
8   Bundesliga (AUT)    SC Austria Lustenau   
9   Bundesliga (AUT)                SV Ried   
10  Bundesliga (AUT)           TSV Hartberg   
11  Bundesliga (AUT)              WSG Tirol   
12  Bundesliga (AUT)           Rapid Vienna   
13  Bundesliga (AUT)         Austria Vienna   
14  Bundesliga (AUT)  SK Austria Klagenfurt   
16  Bundesliga (AUT)         Wolfsberger AC   
21  Bundesliga (AUT)      Red Bull Salzburg   

                                              link  
1             /sk-sturm-graz/startseite/verein/122  
2                      /lask/startseite/verein/413  
6               /scr-altach/startseite/verein/3551  
8       /sc-austria-lustenau/startseite/verein/541  
9                   /sv-ried/startseite/verein/266  
10            /tsv-hartberg/startseite/verein/4467  
11               

                    league                     teams  \
1   Saudi Pro League (KSA)   Al-Shabab Club (Riyadh)   
3   Saudi Pro League (KSA)                Ettifaq FC   
5   Saudi Pro League (KSA)             Al-Taawoun FC   
7   Saudi Pro League (KSA)               Al-Wehda FC   
8   Saudi Pro League (KSA)                  Damac FC   
9   Saudi Pro League (KSA)                    Al-Tai   
10  Saudi Pro League (KSA)               Al-Fayha FC   
11  Saudi Pro League (KSA)                 Abha Club   
12  Saudi Pro League (KSA)                Al-Khaleej   
13  Saudi Pro League (KSA)               Al-Raed SFC   
14  Saudi Pro League (KSA)               Al-Batin FC   
15  Saudi Pro League (KSA)              Al-Adalah FC   
16  Saudi Pro League (KSA)  Al-Ittihad Club (Jeddah)   
19  Saudi Pro League (KSA)               Al-Fateh SC   
24  Saudi Pro League (KSA)         Al-Hilal Saudi FC   
25  Saudi Pro League (KSA)               Al-Nassr FC   

                                           link

In [11]:
print(teams_transfermkt)

                  league                                               link  \
4   Premier League (ENG)           /manchester-united/startseite/verein/985   
7   Premier League (ENG)             /west-ham-united/startseite/verein/379   
8   Premier League (ENG)             /leicester-city/startseite/verein/1003   
9   Premier League (ENG)                 /aston-villa/startseite/verein/405   
10  Premier League (ENG)     /wolverhampton-wanderers/startseite/verein/543   
..                   ...                                                ...   
15        A-League (AUS)         /wellington-phoenix/startseite/verein/8445   
19        A-League (AUS)         /melbourne-city-fc/startseite/verein/25580   
20        A-League (AUS)                  /sydney-fc/startseite/verein/8054   
21        A-League (AUS)             /adelaide-united/startseite/verein/875   
22        A-League (AUS)  /western-sydney-wanderers/startseite/verein/36100   

                       teams  
4          Mancheste

In [12]:
teams_transfermkt.to_csv(r'C:/Users/Dell/Downloads/Work/TTDL/Đồ án/Data/Obtaining_players_and_teams/teams_transfermarket.csv',index=False)

In [15]:
transfermkt_teams = pd.read_csv('C:/Users/Dell/Downloads/Work/TTDL/Đồ án/Data/Obtaining_players_and_teams/teams_transfermarket.csv')

tm = pd.DataFrame(columns = ['league', 'link', 'teams'])
for index, row in transfermkt_teams.iterrows():
    if ">" not in row['teams']:
        tm = tm.append(row, ignore_index=True)
print(len(list(tm['teams'])))

374


In [16]:
tm.to_csv(r'C:/Users/Dell/Downloads/Work/TTDL/Đồ án/Data/Obtaining_players_and_teams/teams_transfermarket.csv',index=False)

In [29]:
transfermkt_teams = pd.read_csv('C:/Users/Dell/Downloads/Work/TTDL/Đồ án/Data/Obtaining_players_and_teams/teams_transfermarket.csv')

li = list(transfermkt_teams['teams'])
print(li)

['Manchester United', 'West Ham United', 'Leicester City', 'Aston Villa', 'Wolverhampton Wanderers', 'Southampton FC', 'Brighton &amp; Hove Albion', 'Everton FC', 'Nottingham Forest', 'Leeds United', 'Crystal Palace', 'Fulham FC', 'AFC Bournemouth', 'Brentford FC', 'Liverpool FC', 'Newcastle United', 'Manchester City', 'Arsenal FC', 'Tottenham Hotspur', 'Chelsea FC', 'Borussia Mönchengladbach', 'VfL Wolfsburg', 'TSG 1899 Hoffenheim', '1.FC Union Berlin', 'FC Augsburg', 'VfB Stuttgart', '1.FSV Mainz 05', '1.FC Köln', 'Hertha BSC', 'FC Schalke 04', 'VfL Bochum', 'SV Werder Bremen', 'Eintracht Frankfurt', 'SC Freiburg', 'Borussia Dortmund', 'Bayer 04 Leverkusen', 'Bayern Munich', 'RB Leipzig', 'Real Sociedad', 'Villarreal CF', 'Real Betis Balompié', 'Athletic Bilbao', 'Valencia CF', 'Sevilla FC', 'Celta de Vigo', 'CA Osasuna', 'Girona FC', 'Rayo Vallecano', 'RCD Mallorca', 'UD Almería', 'Real Valladolid CF', 'Cádiz CF', 'Elche CF', 'RCD Espanyol Barcelona', 'Getafe CF', 'Atlético de Madri

In [32]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

print(transfermkt_teams)

                            league  \
0             Premier League (ENG)   
1             Premier League (ENG)   
2             Premier League (ENG)   
3             Premier League (ENG)   
4             Premier League (ENG)   
5             Premier League (ENG)   
6             Premier League (ENG)   
7             Premier League (ENG)   
8             Premier League (ENG)   
9             Premier League (ENG)   
10            Premier League (ENG)   
11            Premier League (ENG)   
12            Premier League (ENG)   
13            Premier League (ENG)   
14            Premier League (ENG)   
15            Premier League (ENG)   
16            Premier League (ENG)   
17            Premier League (ENG)   
18            Premier League (ENG)   
19            Premier League (ENG)   
20                Bundesliga (GER)   
21                Bundesliga (GER)   
22                Bundesliga (GER)   
23                Bundesliga (GER)   
24                Bundesliga (GER)   
25          